In [1]:
import sys
sys.path.append('..')
from osp import *

df_meta = get_corpus_metadata(periodize_by=30).query('year>=1900 and year<2020')
df_meta_lit = df_meta.query('discipline == "Literature"')
df_meta_phil = df_meta.query('discipline == "Philosophy"')

In [2]:
words = get_non_content_words()
len(words)

# wordsets_words=get_wordsets(n_words=500, wordsets=['non_content'])
wordsets_words={'non_content':words}

all_data = {}
for wordset, words in wordsets_words.items():
    all_data[wordset] = get_counts_wordset(words)

finding word freqs: 100%|██████████| 780/780 [00:00<00:00, 2987723.40it/s]


In [34]:
all_df_preds, all_df_feats = classify_all_data(
    all_data, 
    classify_by=['journal'], 
    data_meta=df_meta, 
    sample_size=None,
    verbose=False
)
all_df_preds

classifying pairwise samples for journal: 100%|██████████| 210/210 [00:06<00:00, 33.54it/s]


,true_label,pred_label,test_label,confidence,correct,accuracy,support,test_group,test_group_type,wordset,classify_by
id,,,,,,,,,,,
phil/10.2307/20013351,Erkenntnis (1975-),Erkenntnis (1975-),Erkenntnis (1975-) / The Review of English Stu...,0.501738,True,0.947368,38,Erkenntnis (1975-) vs. The Review of English S...,journal,non_content,journal
phil/10.2307/26706558,Erkenntnis (1975-),Erkenntnis (1975-),Erkenntnis (1975-) / The Review of English Stu...,0.504834,True,0.947368,38,Erkenntnis (1975-) vs. The Review of English S...,journal,non_content,journal
lit/519198,The Review of English Studies,The Review of English Studies,Erkenntnis (1975-) / The Review of English Stu...,0.505255,True,0.947368,38,Erkenntnis (1975-) vs. The Review of English S...,journal,non_content,journal
lit/508240,The Review of English Studies,The Review of English Studies,Erkenntnis (1975-) / The Review of English Stu...,0.505137,True,0.947368,38,Erkenntnis (1975-) vs. The Review of English S...,journal,non_content,journal
lit/511910,The Review of English Studies,The Review of English Studies,Erkenntnis (1975-) / The Review of English Stu...,0.501832,True,0.947368,38,Erkenntnis (1975-) vs. The Review of English S...,journal,non_content,journal
...,...,...,...,...,...,...,...,...,...,...,...
lit/462465,PMLA,Modern Philology,Modern Philology / PMLA,0.501157,False,0.263158,38,Modern Philology vs. PMLA,journal,non_content,journal
lit/1261506,PMLA,Modern Philology,Modern Philology / PMLA,0.500279,False,0.263158,38,Modern Philology vs. PMLA,journal,non_content,journal
lit/461232,PMLA,Modern Philology,Modern Philology / PMLA,0.500856,False,0.263158,38,Modern Philology vs. PMLA,journal,non_content,journal


In [35]:
# all_df_preds.support.describe()

In [39]:
df_preds_sum = all_df_preds.groupby(['wordset','test_label']).mean(numeric_only=True).sort_values('accuracy',ascending=False).reset_index()
grps = [x.split(' / ') for x in df_preds_sum['test_label']]
df_preds_sum['group1'], df_preds_sum['group2'] = zip(*grps)
df_preds_sum['period1'], df_preds_sum['discipline1'] = zip(*[(x.split(' ',1) if ' ' in x else ("?",x)) for x in df_preds_sum['group1']])
df_preds_sum['period2'], df_preds_sum['discipline2'] = zip(*[(x.split(' ',1) if ' ' in x else ("?",x)) for x in df_preds_sum['group2']])
df_preds_sum['confusion'] = 1 - df_preds_sum['accuracy']

def categorize_row(row):
    out = ''
    if row.period1 == row.period2:
        out += 'within periods'
    else:
        out += 'across periods'
    if row.discipline1 == row.discipline2:
        out += ' and within disciplines'
    else:
        out += ' and across disciplines'
    return out
df_preds_sum['categorization'] = df_preds_sum.apply(categorize_row, axis=1)
df_preds_sum['correct_fpk'] = (df_preds_sum['accuracy'] * 100).apply(int)
df_preds_sum[['period1','period2','discipline1','discipline2','categorization']]
df_preds_sum.categorization.value_counts()

categorization
across periods and across disciplines    183
within periods and across disciplines     26
across periods and within disciplines      1
Name: count, dtype: int64

In [40]:
df_preds_sum.groupby(['categorization','wordset']).mean(numeric_only=True).sort_values('accuracy',ascending=False).round(2)

,,confidence,correct,accuracy,support,confusion,correct_fpk
categorization,wordset,,,,,,
within periods and across disciplines,non_content,0.53,0.75,0.75,38.0,0.25,74.42
across periods and across disciplines,non_content,0.52,0.72,0.72,38.0,0.28,71.15
across periods and within disciplines,non_content,0.50,0.58,0.58,38.0,0.42,57.00


In [42]:
df_preds_sum

,wordset,test_label,confidence,correct,accuracy,support,group1,group2,period1,discipline1,period2,discipline2,confusion,categorization,correct_fpk
0,non_content,Erkenntnis (1975-) / The Review of English Stu...,0.503038,0.947368,0.947368,38.0,Erkenntnis (1975-),The Review of English Studies,Erkenntnis,(1975-),The,Review of English Studies,0.052632,across periods and across disciplines,94
1,non_content,Ethics / The Review of English Studies,0.503078,0.921053,0.921053,38.0,Ethics,The Review of English Studies,?,Ethics,The,Review of English Studies,0.078947,across periods and across disciplines,92
2,non_content,The Journal of Philosophy / The Review of Engl...,0.503068,0.921053,0.921053,38.0,The Journal of Philosophy,The Review of English Studies,The,Journal of Philosophy,The,Review of English Studies,0.078947,within periods and across disciplines,92
3,non_content,The Journal of Philosophy / The Modern Languag...,0.503269,0.921053,0.921053,38.0,The Journal of Philosophy,The Modern Language Review,The,Journal of Philosophy,The,Modern Language Review,0.078947,within periods and across disciplines,92
4,non_content,Canadian Journal of Philosophy / The Review of...,0.504125,0.921053,0.921053,38.0,Canadian Journal of Philosophy,The Review of English Studies,Canadian,Journal of Philosophy,The,Review of English Studies,0.078947,across periods and across disciplines,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,non_content,PMLA / The Modern Language Review,0.502672,0.447368,0.447368,38.0,PMLA,The Modern Language Review,?,PMLA,The,Modern Language Review,0.552632,across periods and across disciplines,44
206,non_content,Philosophy and Phenomenological Research / Syn...,0.501880,0.447368,0.447368,38.0,Philosophy and Phenomenological Research,Synthese,Philosophy,and Phenomenological Research,?,Synthese,0.552632,across periods and across disciplines,44
207,non_content,The Modern Language Review / The Review of Eng...,0.500819,0.421053,0.421053,38.0,The Modern Language Review,The Review of English Studies,The,Modern Language Review,The,Review of English Studies,0.578947,within periods and across disciplines,42
208,non_content,Proceedings of the Aristotelian Society / The ...,0.500316,0.315789,0.315789,38.0,Proceedings of the Aristotelian Society,The Journal of Philosophy,Proceedings,of the Aristotelian Society,The,Journal of Philosophy,0.684211,across periods and across disciplines,31


In [43]:
# import plotnine as p9
# p9.options.figure_size = (10, 10)
# p9.options.dpi = 300

# df_preds_sum2 = df_preds_sum.copy()
# df_preds_sum2['group1'] = df_preds_sum['group2']
# df_preds_sum2['group2'] = df_preds_sum['group1']
# df_preds_sum3 = pd.concat([df_preds_sum, df_preds_sum2])

# fig = p9.ggplot(df_preds_sum3, p9.aes(x='group1', y='group2', fill='accuracy')) + \
#     p9.geom_tile() + \
#     p9.geom_text(p9.aes(label='correct_fpk'), color='black', size=10) + \
#     p9.scale_fill_gradient(low='white', high='blue') + \
#     p9.theme_minimal() + \
#     p9.labs(x='Group 1', y='Group 2', fill='Accuracy') + \
#     p9.theme(axis_text_x=p9.element_text(angle=45))
# fig


In [ ]:
def plot_network(df_preds_sum, wordset, cutoff=.9, color_by='century_journal'):

    import networkx as nx
    import matplotlib.pyplot as plt

    G = nx.Graph()
    x = ' / '

    def make_label(period, discipline):
        return f"{period} {discipline}"

    # First, build the graph and collect periods for each node
    node_periods = {}
    node_disciplines = {}

    for i, row in df_preds_sum[df_preds_sum['wordset'] == wordset].set_index('test_label').iterrows():
        a, b = i.split(x)
        a1, a2 = a.split(' ',1) if ' ' in a else ("?",a)
        b1, b2 = b.split(' ',1) if ' ' in b else ("?",b)
        label_a = make_label(a1, a2)
        label_b = make_label(b1, b2)
        # Store period for node coloring
        node_periods[label_a] = a1
        node_periods[label_b] = b1
        node_disciplines[label_a] = a2
        node_disciplines[label_b] = b2
        if row['accuracy'] >= cutoff:
            continue
        # Store edge, will scale width by accuracy
        G.add_edge(label_a, label_b, weight=row['accuracy'])

    # Create a color map for periods
    periods = sorted({period for period in node_periods.values()}) + ["?"]
    disciplines = sorted({discipline for discipline in node_disciplines.values()})
    color_map = dict(zip(periods, plt.cm.Set1.colors[:len(periods)]))
    # color_map = {}
    color_map.update(dict(zip(disciplines, plt.cm.Set2.colors[:len(disciplines)])))

    # Assign colors to nodes based on period
    node_colors = [color_map[node_periods.get(node, "?")] for node in G.nodes()]

    # Prepare edge widths from min to max -> scale to between 1 and 10
    edges = G.edges(data=True)
    edge_weights = [d['weight'] for _, _, d in edges]
    edge_weights_neg = [1-w for w in edge_weights]
    if edge_weights_neg:
        min_w = min(edge_weights_neg)
        max_w = max(edge_weights_neg)
        if max_w > min_w:
            # scale each w from [min_w, max_w] to [1, 10]
            edge_widths = [1 + 9 * ((w - min_w) / (max_w - min_w)) for w in edge_weights_neg]
        else:
            # all weights are the same, use a fixed width
            edge_widths = [5 for _ in edge_weights]
    else:
        edge_widths = []

    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(12, 8))
    nodes = nx.draw_networkx_nodes(G, pos, node_color=node_colors, cmap=plt.cm.Set1)

    nx.draw_networkx_edges(G, pos, edge_color='gray', width=edge_widths)
    nx.draw_networkx_labels(G, pos, font_size=10)

    # Draw edge labels: accuracy to 2 decimal places
    edge_labels = {(u, v): f"{d['weight']:.2f}" for u, v, d in G.edges(data=True)}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    plt.title("Pairwise classification relationships (period as discipline)\n(Node color: period, edge width = accuracy)")
    # Custom legend for periods
    from matplotlib.patches import Patch
    # handles = [Patch(color=color_map[period], label=period) for period in periods]
    handles = [Patch(color=color_map[period], label=period) for period in periods]
    plt.legend(handles=handles, title="Period")
    plt.axis("off")
    plt.show()


In [51]:
plot_network(df_preds_sum, 'non_content', cutoff=.7, color_by='century_journal')

KeyError: 'The'